In [106]:
import impact as                             impt
from impact.parsers import Parser as parser

impt.settings.perform_curve_fit = False
impt.settings.outlier_cleaning_flag = False
impt.settings.verbose = False
impt.settings.live_calculations = False
impt.settings.use_filtered_data = True
impt.settings.blank_subtract = False

import numpy as np
from scipy.signal import savgol_filter as savgol
from scipy.interpolate import InterpolatedUnivariateSpline as interp
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.optimize import curve_fit
import pandas as pd
import copy
import pickle

import plotly as py
import plotly.graph_objs as go
import plotly.io as pio
pio.templates.default = "plotly_white"
from plotly.subplots import make_subplots
import sys
if 'ipykernel' in sys.modules:
    from plotly.offline import init_notebook_mode
    from plotly.offline import iplot as plot
    from IPython.display import HTML
    HTML(
        """
        <script>
            var waitForPlotly = setInterval( function() {
                if( typeof(window.Plotly) !== "undefined" ){
                    MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
                    MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
                    clearInterval(waitForPlotly);
                }}, 250 );
        </script>
        """
    )
    init_notebook_mode(connected=True)

colors = {'blues':['rgb(164,199,228)','rgb(119,171,214)','rgb(73,143,201)',
                   'rgb(49,113,165)','rgb(36,82,120)','rgb(23,52,76)'],
          'greens':['rgb(172,220,173)','rgb(130,203,133)','rgb(89,185,92)',
                    'rgb(63,152,66)','rgb(46,110,48)','rgb(28,67,29)'],
          'reds':['rgb(230,174,176)','rgb(218,133,137)','rgb(205,92,98)',
                  'rgb(187,57,64)','rgb(146,45,50)','rgb(104,32,35)'],
          'pinks':['rgb(215,181,210)','rgb(195,144,188)','rgb(175,107,165)',
                   'rgb(147,79,137)','rgb(110,59,102)','rgb(74,40,69)'],
          'browns':['rgb(216,197,184)','rgb(196,168,148)','rgb(176,139,112)',
                    'rgb(150,111,83)','rgb(115,85,63)','rgb(80, 59, 44)'],
          'oranges': ['rgb(255,206,146)','rgb(255,189,109)','rgb(255,173,73)',
                      'rgb(255,157,36)','rgb(255, 140, 0)','rgb(219,120,0)'],
          'cyans': ['rgb(177,222,219)','rgb(139,205,201)','rgb(100,189,183)',
                    'rgb(70,163,157)','rgb(53,124,120)','rgb(37,87,84)'],
          'yellows': ['rgb(244,239,186)','rgb(239,231,151)','rgb(233,224,116)',
                      'rgb(228,216,82)','rgb(223,208,47)','rgb(200,185,31)',],
          'blacks':['rgb(182,182,182)','rgb(146,146,146)','rgb(109,109,109)',
                    'rgb(73,73,73)','rgb(36,36,36)','rgb(0,0,0)']}

In [107]:
font_defaults = dict(family='Myriad Pro', color='black', size=18)
axis_defaults = dict(ticks='outside', ticklen=6, tickangle=0, nticks=10, tickcolor='black', tickfont=font_defaults,
                     showline=True, linewidth=1, linecolor='black', mirror=True, zeroline=False,
                     title_font=font_defaults, title_standoff=2, showgrid=True)
stabilities = ['bistable', 'tet_stable', 'lac_stable', 'unstable']
stability_titles = {'bistable':'Bistable', 'tet_stable': 'Monostable Tet',
                    'lac_stable': 'Monostable Lac', 'unstable': 'Unstable'}
stability_colors = {'bistable': colors['blues'][2],
                    'tet_stable': colors['reds'][2],
                    'lac_stable': colors['greens'][2],
                    'unstable': colors['blacks'][2]}

def bistability_assigner(row, bistable_threshold, unstable_threshold):
    
    if row['iptg_transfers_stable']>=bistable_threshold and row['atc_transfers_stable']>=bistable_threshold:
        return 'bistable'
    
    if row['iptg_transfers_stable']<unstable_threshold and row['atc_transfers_stable']<unstable_threshold:
        return 'unstable'
    
    if row['iptg_transfers_stable'] > row['atc_transfers_stable']:
        return 'tet_stable'
    
    if row['atc_transfers_stable'] > row['iptg_transfers_stable']:
        return 'lac_stable'
    
    
def get_bistability_dfs(expt_dict=dict(), thresholds={'GFP': 100, 'mCherry': 10}, 
                        bistable_threshold=3, unstable_threshold=3):
    data_df = {'iptg': None, 'atc': None}
    analyte = {'iptg':'mCherry', 'atc':'GFP'}
    autofl_reps = {'iptg':'blank', 'atc':'lac'}
    days = ['d0', 'd1', 'd2','d3']
    
    for init_state in init_states:
        strains = []
        rep_ids = []
        state_vals = {day:[] for day in days}

        for plate in plates:
            for i, day in enumerate(days):
                expt = expt_dict[init_state][plate][day]
                autofl_rep = [rep for rep in expt.replicate_trials 
                              if rep.trial_identifier.strain.name.lower()==autofl_reps[init_state]][0]

                autofl_avg = autofl_rep.avg.analyte_dict[analyte[init_state]].data_vector[-1]
                autofl_std = autofl_rep.std.analyte_dict[analyte[init_state]].data_vector[-1]
                
                if (init_state=='iptg') and (np.isnan(autofl_std) or (3*autofl_std < 0.1*autofl_avg)):
                    autofl_std = 0.1/3*autofl_avg
                    
                autofl_threshold = autofl_avg + 3 * autofl_std
                strain_names = [rep.trial_identifier.strain.name for rep in expt.replicate_trials 
                                if rep.trial_identifier.strain.name.lower() not in ['lac', 'blank', '0000']]
                strain_names = sorted(strain_names, key=lambda x:(x[2:], x[:2]))

                for strain_name in strain_names:
                    rep = [rep for rep in expt.replicate_trials if rep.trial_identifier.strain.name==strain_name][0]
                    trial_dict = rep.single_trial_dict

                    for j, rep_id in enumerate(sorted(list(trial_dict))):
                        analyte_dict = trial_dict[rep_id].analyte_dict
                        raw_reporter = analyte_dict[analyte[init_state]].data_vector[-1]
                        reporter = raw_reporter - autofl_threshold

                        if reporter < thresholds[analyte[init_state]]:
                            reporter = 0

                        state_vals[day].append(reporter)

                        if i==0:
                            rep_ids.append(rep_id)
                            strains.append(strain_name)

        data_dict = {'strain': strains, 'rep': rep_ids}
        data_dict = {**data_dict, **{'state_'+day:state_vals[day] for day in days}}
        data_df[init_state] = pd.DataFrame(data=data_dict)
        data_df[init_state]['rep'] = data_df[init_state]['rep'].replace(['4','5','6'],['1','2','3'])
        data_df[init_state] = data_df[init_state].sort_values(by=['strain', 'rep'], 
                                                                        ascending=[True,True],
                                                                        key=lambda x:x.apply(lambda y:(y[:2], y[2:])))
        data_df[init_state] = data_df[init_state].set_index(['strain','rep'])
        temp = (data_df[init_state][['state_'+day for day in days[1:]]]>0).idxmin(axis=1).apply(lambda x: int(x[-1]))
        data_df[init_state][init_state+'_transfers_stable'] = (temp * (data_df[init_state][['state_'+day for day in days[1:]]]==0).any(axis=1)).replace(0,6).apply(lambda x:x-1)
        del data_dict
        
    st_bistability_df = pd.concat([data_df[init_state][[init_state+'_transfers_stable']] for init_state in init_states], axis=1)
    st_bistability_df['stability'] = st_bistability_df.apply(lambda x: bistability_assigner(x, bistable_threshold=bistable_threshold, unstable_threshold=unstable_threshold), axis=1)
    strains = []
    stability = []

    for strain in sorted(list(set(st_bistability_df.index.get_level_values(0))), key=lambda x:(x[2:], x[:2])):
        strains.append(strain)
        if (st_bistability_df.loc[strain]['stability']=='bistable').sum()>=3:
            stability.append('bistable')
        elif st_bistability_df.loc[strain]['stability'].isin(['bistable', 'tet_stable']).all():
            stability.append('tet_stable')
        elif st_bistability_df.loc[strain]['stability'].isin(['bistable', 'lac_stable']).all():
            stability.append('lac_stable')
        else:
            stability.append('unstable')

    temp_dict = {'strain':strains, 'stability': stability}
    rep_bistability_df = pd.DataFrame(data=temp_dict)
    rep_bistability_df = rep_bistability_df.set_index('strain')
    return data_df, st_bistability_df, rep_bistability_df


In [110]:
with open('pickled_data/stability_data.pickle', 'rb') as handle:
    expt_dict=pickle.load(handle)

In [138]:
deg_tags = ['No Tag', 'DAS Tag', 'DAS+2 Tag', 'LAA Tag']
deg_codes = {'0': '*', '1': 'DAS', '2':'DAS+2', '3':'LAA'}
rbs_codes = {'1':'A', '2':'B', '3': 'C', '4':'D', '5':'E'}

mche_df = pd.read_csv('misc_excel_data/mche_df.csv')
mche_df['tet_code'] = mche_df['tet_code'].astype('str')
mche_df = mche_df.set_index('tet_code')
mche_df = mche_df.loc[['21', '22', '30', '32', '33', '43', '50', '51', '52', '53']]
mche_df['deg_class'] = pd.qcut(mche_df['deg_mean'], 3, labels=['Low', 'Mid', 'High'])
mche_df['deg_tag'] = mche_df.index.map(lambda x:deg_tags[int(x[1])])

gfp_df = pd.read_csv('misc_excel_data/gfp_df.csv')
gfp_df['lac_code'] = gfp_df['lac_code'].astype('str')
gfp_df = gfp_df.set_index('lac_code')
gfp_df = gfp_df.loc[['10', '20', '21', '30', '33', '40', '42', '43', '52', '53']]
gfp_df['deg_class'] = pd.qcut(gfp_df['deg_mean'], 3, labels=['Low', 'Mid', 'High'])
gfp_df['deg_tag'] = gfp_df.index.map(lambda x:deg_tags[int(x[1])])

In [198]:
thresholds = {'GFP': 25, 'mCherry':15}

bistable_threshold = 3
unstable_threshold = 3
    
data_df, st_bistability_df, rep_bistability_df = get_bistability_dfs(expt_dict, thresholds, 
                                                                    bistable_threshold, unstable_threshold)

In [199]:
bistability_dfs = [st_bistability_df, rep_bistability_df]
titles = ['Single Trials', 'Replicates']

for n,bistability_df in enumerate(bistability_dfs):
    fig = make_subplots(rows=4, cols=4, horizontal_spacing=0.005, vertical_spacing=0.005)
    
    for i, lac_deg in enumerate(['0','1','2','3']):
        for j, tet_deg in enumerate(['0','1','2','3']):
            temp_df = bistability_df[bistability_df.index.get_level_values(0).map(lambda x:(x[1], x[3]))==(lac_deg,tet_deg)]
            temp_dict = {stability:(temp_df['stability']==stability).sum() for stability in stabilities}
            cumulative = np.cumsum([temp_dict[key]/len(temp_df) for key in temp_dict])

            if lac_deg=='0' and tet_deg=='0':
                showlegend = True
            else:
                showlegend = False

            for k, stability in enumerate(stabilities):
                fig.add_trace(go.Scatter(y=np.linspace(0,1,10), x=[cumulative[k]]*10,
                                         mode='lines', line=dict(width=0), 
                                         name=legend_dict[stability], showlegend=showlegend, legendgroup=stability,
                                         fillcolor=stability_colors[stability], fill='tonextx'),
                              row=4-j, col=i+1)

                if temp_dict[stability]!=0:
                    fig.add_annotation(x=(cumulative - np.diff(cumulative, prepend=0)/2)[k], y=0.5,
                                       text=str(int(np.round(temp_dict[stability]*100/len(temp_df), decimals=0)))+'%',
                                       showarrow=False, row=4-int(j), col=int(i)+1)
            fig.add_annotation(x=0.5, y=0.8, text='n = '+str(len(temp_df)), showarrow=False,row=4-j, col=i+1)

            if i==0:
                fig.update_yaxes(title=deg_tags[int(tet_deg)], row=4-j, col=i+1)
            if j==0:
                fig.update_xaxes(title=deg_tags[int(lac_deg)], row=4-j, col=i+1)

    fig.update_layout(width=750, height=650, legend_font=font_defaults, title=titles[n], title_font=font_defaults)
    fig.update_xaxes({**axis_defaults, 
                      **dict(title_standoff=10, titlefont=dict(family='Myriad Pro', size=24, color='black'),
                             showticklabels=False, range=(0,1), ticklen=0)}) 
    fig.update_yaxes({**axis_defaults, 
                      **dict(title_standoff=0, titlefont=dict(family='Myriad Pro', size=24, color='black'),
                             showticklabels=False, range=(0,1), ticklen=0)})
    fig.update_annotations(font=font_defaults)
    plot(fig)
    if n==0:
        pio.write_image(fig,"figures\Robustness_Tags_SingleTrials.svg",format='svg')

In [202]:
bistability_dfs = [st_bistability_df, rep_bistability_df]
titles = ['Single Trials', 'Replicates']

for n,bistability_df in enumerate(bistability_dfs):
    fig = make_subplots(rows=3, cols=3, horizontal_spacing=0.01, vertical_spacing=0.01)
    
    for i, lac_deg in enumerate(['Low', 'Mid', 'High']):
        for j, tet_deg in enumerate(['Low', 'Mid', 'High']):
            
            temp_df = bistability_df[bistability_df.index.get_level_values(0).map(lambda x:(gfp_df.loc[x[:2]]['deg_class'], 
                                                                                            mche_df.loc[x[2:]]['deg_class']))==(lac_deg, tet_deg)]
            temp_dict = {stability:(temp_df['stability']==stability).sum() for stability in stabilities}
            cumulative = np.cumsum([temp_dict[key]/len(temp_df) for key in temp_dict])

            if i==0 and j==0:
                showlegend = True
            else:
                showlegend = False

            for k, stability in enumerate(stabilities):
                fig.add_trace(go.Scatter(y=np.linspace(0,1,10), x=[cumulative[k]]*10,
                                         mode='lines', line=dict(width=0), 
                                         name=legend_dict[stability], showlegend=showlegend, legendgroup=stability,
                                         fillcolor=stability_colors[stability], fill='tonextx'),
                              row=3-j, col=i+1)

                if temp_dict[stability]!=0:
                    fig.add_annotation(x=(cumulative - np.diff(cumulative, prepend=0)/2)[k], y=0.5,
                                       text=str(int(np.round(temp_dict[stability]*100/len(temp_df), decimals=0)))+'%',
                                       showarrow=False, row=3-int(j), col=int(i)+1)
            fig.add_annotation(x=0.5, y=0.8, text='n = '+str(len(temp_df)), showarrow=False,row=3-j, col=i+1)

            if i==0:
                fig.update_yaxes(title=tet_deg, row=3-j, col=i+1)
            if j==0:
                fig.update_xaxes(title=lac_deg, row=3-j, col=i+1)

    fig.update_layout(width=675, height=600, legend_font=font_defaults, title=titles[n], title_font=font_defaults)
    fig.update_xaxes({**axis_defaults, 
                      **dict(title_standoff=10, titlefont=dict(family='Myriad Pro', size=24, color='black'),
                             showticklabels=False, range=(0,1), ticklen=0)}) 
    fig.update_yaxes({**axis_defaults, 
                      **dict(title_standoff=0, titlefont=dict(family='Myriad Pro', size=24, color='black'),
                             showticklabels=False, range=(0,1), ticklen=0)})
    fig.update_annotations(font=font_defaults)
    plot(fig)
    
    if n==0:
        pio.write_image(fig,"figures\Robustness_RateBins_SingleTrials.svg",format='svg')
    if n==1:
        pio.write_image(fig,"figures\Robustness_RateBins_Replicates.svg",format='svg')
        


In [203]:
bistability_dfs = [st_bistability_df, rep_bistability_df]
titles = ['Single Trials', 'Replicates']

for n, bistability_df in enumerate(bistability_dfs):
    fig = make_subplots(rows=1, cols=3, horizontal_spacing=0.1, vertical_spacing=0.1)

    for i, deg_class in enumerate(['Low', 'Mid', 'High']):
        temp_df = bistability_df[bistability_df.index.get_level_values(0).map(lambda x:gfp_df.loc[x[:2]]['deg_class'])==deg_class]
        deg_labels = []
        bistable_counts = []
        lac_codes = sorted(temp_df.index.get_level_values(0).map(lambda x:x[:2]).unique(), 
                           key=lambda x:gfp_df.loc[x]['deg_mean'])
        for lac_code in lac_codes:
            temp_temp_df = temp_df[temp_df.index.get_level_values(0).map(lambda x:x[:2]==lac_code)]
            try:
                bistable_counts.append(temp_temp_df['stability'].value_counts(True)['bistable'])
            except KeyError:
                bistable_counts.append(0)
        fig.add_trace(go.Bar(x=[rbs_codes[code[0]]+'-'+deg_codes[code[1]] for code in lac_codes],
                             y=bistable_counts,
                             marker=dict(color=stability_colors['bistable']), showlegend=False,
                             orientation='v'), row=1, col=i+1)

        fig.update_layout(width=700,height=350, bargap=0.5, title = titles[n]+'_'+'Lac', title_font = font_defaults)
        fig.update_xaxes({**axis_defaults, **dict(showgrid=False, tickangle=-90)})
        fig.update_yaxes({**axis_defaults, **dict(showgrid=False, range=(0,1))})

    plot(fig)
    if n==0:
        pio.write_image(fig,"figures\Robustness_Individual_SingleTrials_lacs.svg",format='svg')
    if n==1:
        pio.write_image(fig,"figures\Robustness_Individual_Replicates_lacs.svg",format='svg')
    
    
    fig = make_subplots(rows=3, cols=1, horizontal_spacing=0.1, vertical_spacing=0.1)
    
    for i, deg_class in enumerate(['High', 'Mid', 'Low']):
        temp_df = bistability_df[bistability_df.index.get_level_values(0).map(lambda x:mche_df.loc[x[2:]]['deg_class'])==deg_class]
        deg_labels = []
        bistable_counts = []
        tet_codes = sorted(temp_df.index.get_level_values(0).map(lambda x:x[2:]).unique(), 
                           key=lambda x:mche_df.loc[x]['deg_mean'])
        for tet_code in tet_codes:
            temp_temp_df = temp_df[temp_df.index.get_level_values(0).map(lambda x:x[2:]==tet_code)]
            try:
                bistable_counts.append(temp_temp_df['stability'].value_counts(True)['bistable'])
            except KeyError:
                bistable_counts.append(0)

        fig.add_trace(go.Bar(y=[rbs_codes[code[0]]+'-'+deg_codes[code[1]] for code in tet_codes],
                             x=bistable_counts,
                             marker=dict(color=stability_colors['bistable']), showlegend=False,
                             orientation='h'), row=i+1, col=1)

        fig.update_layout(width=330,height=775, bargap=0.5, title = titles[n]+'_'+'Tet', title_font = font_defaults)
        fig.update_yaxes({**axis_defaults, **dict(showgrid=False, tickangle=0)})
        fig.update_xaxes({**axis_defaults, **dict(showgrid=False, range=(0,1), tickangle=-90)})

    plot(fig)
    if n==0:
        pio.write_image(fig,"figures\Robustness_Individual_SingleTrials_tets.svg",format='svg')
    if n==1:
        pio.write_image(fig,"figures\Robustness_Individual_Replicates_tets.svg",format='svg')

In [197]:
rep_bistability_df.to_csv('misc_excel_data/rep_bistability_df.csv')
st_bistability_df.to_csv('misc_excel_data/st_bistability_df.csv')